In [ ]:
from google.colab import files
files.upload()  # Загрузить kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d harlequeen/synthetic-rgbd-images-of-plants
!unzip synthetic-rgbd-images-of-plants.zip

In [ ]:
!pip install OpenEXR pillow

In [ ]:
import OpenEXR
import Imath
import numpy as np
from PIL import Image
import cv2
import imageio.v2 as imageio
from PIL import Image
import numpy as np
from keras.preprocessing.image import img_to_array
import tensorflow.keras.backend as K

from keras.layers import Layer
from keras.layers import Input
from keras.layers import Convolution2D
from keras.layers import Activation, Reshape
from keras.layers import BatchNormalization
from keras.layers import add
from keras.layers import Dropout
from keras.models import Model
from keras.layers import Concatenate
from keras.layers import Conv2DTranspose
from keras.layers import UpSampling2D
from keras.layers import Average
import keras.optimizers
import argparse
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import MaxPooling2D, UpSampling2D


In [ ]:
def unetpp(input_shape, n_labels, kernel=3,pool_size=(2, 2),output_mode="softmax"):
  inputs = Input(shape=input_shape)

  X00 = MaxPooling2D(pool_size)(inputs)
  X00 = Convolution2D(32,(kernel,kernel),padding = "same")(X00)
  X00 = BatchNormalization()(X00)
  X00 = Activation("relu")(X00)

  X10 = MaxPooling2D(pool_size)(X00)
  X10 = Convolution2D(64,(kernel,kernel),padding = "same")(X10)
  X10 = BatchNormalization()(X10)
  X10 = Activation("relu")(X10)

  tmpX10 = UpSampling2D(pool_size)(X10)


  X01 = Convolution2D(32,(kernel,kernel),padding = "same")(Concatenate()([X00,tmpX10]))
  X01 = BatchNormalization()(X01)
  X01 = Activation("relu")(X01)

  X20 = MaxPooling2D(pool_size)(X10)
  X20 = Convolution2D(128,(kernel,kernel),padding = "same")(X20)
  X20 = BatchNormalization()(X20)
  X20 = Activation("relu")(X20)

  tmpX20 = UpSampling2D(pool_size)(X20)
  X11 = Convolution2D(64,(kernel,kernel),padding = "same")(Concatenate()([X10,tmpX20]))
  X11 = BatchNormalization()(X11)
  X11 = Activation("relu")(X11)


  tmpX11 = UpSampling2D(pool_size)(X11)
  X02 = Convolution2D(32,(kernel,kernel),padding = "same")(Concatenate()([X00, X01,tmpX11]))
  X02 = BatchNormalization()(X02)
  X02 = Activation("relu")(X02)

  X30 = MaxPooling2D(pool_size)(X20)
  X30 = Convolution2D(256,(kernel,kernel),padding = "same")(X30)
  X30 = BatchNormalization()(X30)
  X30 = Activation("relu")(X30)

  tmpX30 = UpSampling2D(pool_size)(X30)
  X21 = Convolution2D(128,(kernel,kernel),padding = "same")(Concatenate()([X20,tmpX30]))
  X21 = BatchNormalization()(X21)
  X21 = Activation("relu")(X21)


  tmpX21 = UpSampling2D(pool_size)(X21)
  X12 = Convolution2D(64,(kernel,kernel),padding = "same")(Concatenate()([X10,X11,tmpX21]))
  X12 = BatchNormalization()(X12)
  X12 = Activation("relu")(X12)


  tmpX12 = UpSampling2D(pool_size)(X12)
  X03 = Convolution2D(32,(kernel,kernel),padding = "same")(Concatenate()([X00,X01,X02,tmpX12]))
  X03 = BatchNormalization()(X03)
  X03 = Activation("relu")(X03)


  X40 = MaxPooling2D(pool_size)(X30)
  X40 = Convolution2D(512,(kernel,kernel),padding = "same")(X40)
  X40 = BatchNormalization()(X40)
  X40 = Activation("relu")(X40)

  tmpX40 = UpSampling2D(pool_size)(X40)
  X31 = Convolution2D(128,(kernel,kernel),padding = "same")(Concatenate()([X30,tmpX40]))
  X31 = BatchNormalization()(X31)
  X31 = Activation("relu")(X31)


  tmpX31 = UpSampling2D(pool_size)(X31)
  X22 = Convolution2D(128,(kernel,kernel),padding = "same")(Concatenate()([X20,X21,tmpX31]))
  X22 = BatchNormalization()(X22)
  X22 = Activation("relu")(X22)


  tmpX22 = UpSampling2D(pool_size)(X22)
  X13 = Convolution2D(64,(kernel,kernel),padding = "same")(Concatenate()([X10,X11,X12,tmpX22]))
  X13 = BatchNormalization()(X13)
  X13 = Activation("relu")(X13)

  tmpX13 = UpSampling2D(pool_size)(X13)
  X04 = Convolution2D(32,(kernel,kernel),padding = "same")(Concatenate()([X00,X01,X02,X03,tmpX13]))
  X04 = BatchNormalization()(X04)
  X04 = Activation("relu")(X04)

  out_01 = UpSampling2D(size=(2, 2))(X01 )
  out_02 = UpSampling2D(size=(2,2 ))(X02)
  out_03 = UpSampling2D(size=(2, 2))(X03)
  out_04 = UpSampling2D(size=(2, 2))(X04)

  out_01 = Convolution2D(n_labels, (1, 1), activation='softmax', name='output_01')(out_01)
  out_02 = Convolution2D(n_labels, (1, 1), activation='softmax', name='output_02')(out_02)
  out_03 = Convolution2D(n_labels, (1, 1), activation='softmax', name='output_03')(out_03)
  out_04 = Convolution2D(n_labels, (1, 1), activation='softmax', name='output_04')(out_04)
  outputs = Average(name='final_output')([out_01, out_02, out_03, out_04])

  print("Build decoder done..")

  model = Model(inputs=inputs, outputs=outputs, name="UNet",output_mode="softmax")

  return model


model = unetpp((224,224,9), n_labels=5 ,kernel=3, pool_size=(2,2))

print(model.summary())


Build decoder done..


Model: "UNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 9)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_15          │ (None, 112, 112, 9)    │              0 │ input_layer_3[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_52 (Conv2D)        │ (None, 112, 112, 32)   │          2,624 │ max_pooling2d_15[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_52    │ (None, 112, 112, 32)   │            128 │ conv2d_52[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_47             │ (None, 112, 112, 32)   │              0 │ batch_normalization_5… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_16          │ (None, 56, 56, 32)     │              0 │ activation_47[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_53 (Conv2D)        │ (None, 56, 56, 64)     │         18,496 │ max_pooling2d_16[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_53    │ (None, 56, 56, 64)     │            256 │ conv2d_53[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_48             │ (None, 56, 56, 64)     │              0 │ batch_normalization_5… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_17          │ (None, 28, 28, 64)     │              0 │ activation_48[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_55 (Conv2D)        │ (None, 28, 28, 128)    │         73,856 │ max_pooling2d_17[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_55    │ (None, 28, 28, 128)    │            512 │ conv2d_55[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_50             │ (None, 28, 28, 128)    │              0 │ batch_normalization_5… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_18          │ (None, 14, 14, 128)    │              0 │ activation_50[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├──────────────────────

 Total params: 3,956,180 (15.09 MB)

 Trainable params: 3,952,788 (15.08 MB)

 Non-trainable params: 3,392 (13.25 KB)

None


In [ ]:
def segnet(input_shape, n_labels, kernel=3, pool_size=(2, 2), output_mode="softmax"):
    # encoder
    inputs = Input(shape=input_shape)
    conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    residual = conv_1
    conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Dropout(0.3)(conv_2)
    conv_2 = Activation("relu")(conv_2)
    residual = Convolution2D(64, (kernel, kernel), padding="same")(residual)
    residual = BatchNormalization()(residual)
    conv_2 = add([conv_2,residual])


    pool_1 = MaxPooling2D(pool_size=pool_size, strides=pool_size, padding="same")(conv_2)

    residual = pool_1
    conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Dropout(0.3)(conv_4)
    conv_4 = Activation("relu")(conv_4)
    residual = Convolution2D(128, (kernel, kernel), padding="same")(residual)
    residual = BatchNormalization()(residual)
    conv_4 = add([conv_4,residual])

    pool_2 = MaxPooling2D(pool_size=pool_size, strides=pool_size, padding="same")(conv_4)

    residual = pool_2
    conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Dropout(0.3)(conv_6)
    conv_6 = Activation("relu")(conv_6)
    residual = Convolution2D(256, (kernel, kernel), padding="same")(residual)
    residual = BatchNormalization()(residual)
    conv_6 = add([conv_6,residual])

    pool_3 = MaxPooling2D(pool_size=pool_size, strides=pool_size, padding="same")(conv_6)

    residual = pool_3
    conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Dropout(0.3)(conv_8)
    conv_8 = Activation("relu")(conv_8)
    residual = Convolution2D(512, (kernel, kernel), padding="same")(residual)
    residual = BatchNormalization()(residual)
    conv_8 = add([conv_8,residual])

    pool_4 = MaxPooling2D(pool_size=pool_size, strides=pool_size, padding="same")(conv_8)

    residual = pool_4
    conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Dropout(0.3)(conv_11)
    conv_11 = Activation("relu")(conv_11)
    residual = Convolution2D(512, (kernel, kernel), padding="same")(residual)
    residual = BatchNormalization()(residual)
    conv_11 = add([conv_11,residual])

    pool_5 = MaxPooling2D(pool_size=pool_size, strides=pool_size, padding="same")(conv_11)

    # decoder
    unpool_1 = UpSampling2D(size=pool_size, interpolation="bilinear")(pool_5)


    conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_1)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Dropout(0.3)(conv_14)
    conv_14 = Activation("relu")(conv_14)

    unpool_2 = UpSampling2D(size=pool_size, interpolation="bilinear")(conv_14)

    conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_2)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_19 = Convolution2D(256, (kernel, kernel), padding="same")(conv_17)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Dropout(0.3)(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_3 = UpSampling2D(size=pool_size, interpolation="bilinear")(conv_19)

    conv_20 = Convolution2D(256, (kernel, kernel), padding="same")(unpool_3)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Convolution2D(128, (kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Dropout(0.3)(conv_21)
    conv_21 = Activation("relu")(conv_21)


    unpool_4 = UpSampling2D(size=pool_size, interpolation="bilinear")(conv_21)

    conv_23 = Convolution2D(128, (kernel, kernel), padding="same")(unpool_4)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Convolution2D(64, (kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Dropout(0.3)(conv_24)
    conv_24 = Activation("relu")(conv_24)


    unpool_5 = UpSampling2D(size=pool_size, interpolation="bilinear")(conv_24)

    residual = unpool_5
    conv_25 = Convolution2D(64, (kernel, kernel), padding="same")(unpool_5)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu")(conv_25)

    conv_26 = Convolution2D(n_labels, (1, 1), padding="valid")(conv_25)
    conv_26 = BatchNormalization()(conv_26)
    conv_26 = Reshape(
        (input_shape[0], input_shape[1], n_labels),
        input_shape=(input_shape[0], input_shape[1], n_labels),
    )(conv_26)
    outputs = Activation(output_mode)(conv_26)
    print("Build decoder done..")

    model = Model(inputs=inputs, outputs=outputs, name="SegNet")

    return model


model = segnet((224,224,9), n_labels=5 ,kernel=3, pool_size=(2,2), output_mode="softmax")

print(model.summary())

print("tf.__version__ is", tf.__version__)

Build decoder done..


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "SegNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 9)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_15 (Conv2D)        │ (None, 224, 224, 64)   │          5,248 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_15    │ (None, 224, 224, 64)   │            256 │ conv2d_15[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_15             │ (None, 224, 224, 64)   │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_16 (Conv2D)        │ (None, 224, 224, 64)   │         36,928 │ activation_15[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_16    │ (None, 224, 224, 64)   │            256 │ conv2d_16[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 224, 224, 64)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_17 (Conv2D)        │ (None, 224, 224, 64)   │         36,928 │ activation_15[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_16             │ (None, 224, 224, 64)   │              0 │ dropout[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_17    │ (None, 224, 224, 64)   │            256 │ conv2d_17[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 224, 224, 64)   │              0 │ activation_16[0][0],   │
│                           │                        │                │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 112, 112, 64)   │              0 │ add[0][0]              │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_18 (Conv2D)        │ (None, 112, 112, 128)  │         73,856 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_18    │ (None, 112, 112, 128)  │            512 │ conv2d_18[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_17             │ (None, 112, 112, 128)  │              0 │ batch_normalization_1… │
│ (Activation)         

 Total params: 15,699,289 (59.89 MB)

 Trainable params: 15,688,655 (59.85 MB)

 Non-trainable params: 10,634 (41.54 KB)

None
tf.__version__ is 2.18.0


In [ ]:
def category_label(labels, dims, n_labels):
    x = np.zeros([dims[0], dims[1], n_labels])
    for i in range(dims[0]):
        for j in range(dims[1]):
            f=int(labels[i,j])
            x[i, j, f] = 1
    return x
def colorize(img):
    w=img.shape[0]
    h=img.shape[1]
    z=img.shape[2]
    l=np.zeros((w,h,3))
    for i in range(w):
        for j in range(h):
            if img[i,j,0]==1:
                l[i,j,0]=0
                l[i,j,1]=0
                l[i,j,2]=0
            elif img[i,j,1]==1:
                l[i,j,0]=255
                l[i,j,1]=0
                l[i,j,2]=0
            elif img[i,j,2]==1:
                l[i,j,0]=0
                l[i,j,1]=255
                l[i,j,2]=0
            elif img[i,j,3]==1:
                l[i,j,0]=0
                l[i,j,1]=0
                l[i,j,2]=255
            elif img[i,j,4]==1:
                l[i,j,0]=238
                l[i,j,1]=197
                l[i,j,2]=145
    return l

def class_pixels(img):
    w=img.shape[0]
    h=img.shape[1]
    z=img.shape[2]
    l=np.zeros((w,h,z))
    for i in range(w):
        for j in range(h):
            for f in range(z-1):
                if img[i,j,f]==np.max([img[i,j,0],img[i,j,1],img[i,j,2],img[i,j,3],img[i,j,4]]):
                    l[i,j,f]=1
    return l


In [ ]:
import concurrent.futures
def read_exr(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    header = exr_file.header()
    dw = header['dataWindow']
    size = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

    channels = exr_file.channels("RGB", Imath.PixelType(Imath.PixelType.FLOAT))
    r = np.frombuffer(channels[2], dtype=np.float32).reshape(size[1], size[0])
    g = np.frombuffer(channels[1], dtype=np.float32).reshape(size[1], size[0])
    b = np.frombuffer(channels[0], dtype=np.float32).reshape(size[1], size[0])

    img = np.stack([r, g, b], axis=-1)
    return img
def process_file(index, img_dir, mask_dir, depth_dir, node_dir, dims, n_labels):
    img_path = img_dir[index]
    original_img = read_exr(img_path)
    array_img = original_img 
    array_img[..., 0] = (array_img[..., 0] - 0.0017513) / 0.25150472  # R
    array_img[..., 1] = (array_img[..., 1] - 0.00183881) / 0.2346499  # G
    array_img[..., 2] = (array_img[..., 2] - 0.00178873) / 0.15039594  # B


    depth_path = depth_dir[index]
    depth_img = read_exr(depth_path)

    node_path = node_dir[index]
    node_img = read_exr(node_path)

    res_img = np.concatenate((array_img,depth_img,node_img),axis=-1)
    mask_path = mask_dir[index]
    original_mask = read_exr(mask_path)
    array_mask = category_label(original_mask[:, :, 0], dims, n_labels)

    return res_img, array_mask

def data_gen_small(img_dir, mask_dir, depth_dir, node_dir, liste, batch_size, dims=(224, 224), n_labels=5):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while True:
            ix = np.random.choice(liste, batch_size)
            results = list(executor.map(
                lambda index: process_file(index, img_dir, mask_dir, depth_dir, node_dir, dims, n_labels), ix
            ))

            imgs, labels = zip(*results)
            yield np.array(imgs), np.array(labels)

In [ ]:
import os
rgb=[]
depth=[]
mask=[]
node=[]
for dirs,subdir,files in os.walk('/content/dataset of synthetic rgb-d plants/rgb_map'):
    for file_name in files:
      if file_name.endswith(".exr"):
            path_file=dirs+os.sep+file_name
            depth_file='/content/dataset of synthetic rgb-d plants/depth_map/profondeur_map/'+file_name
            mask_file='/content/dataset of synthetic rgb-d plants/semantic_map/segmentation2_map/'+file_name
            node_file='/content/dataset of synthetic rgb-d plants/nodes_map/internoeuds_map/'+file_name
            rgb.append(path_file)
            depth.append(depth_file)
            mask.append(mask_file)
            node.append(node_file)




liste=np.arange(1,10000)
np.random.shuffle(liste)

train_list=liste[0:4000]
val_list=liste[4000:7000]
test_list=liste[7000:9999]


train_gen = data_gen_small(rgb
,mask,depth,node,liste=train_list,batch_size=32,dims=(224,224),n_labels=5)
val_gen=data_gen_small(rgb
,mask,depth,node,liste=val_list,batch_size=32,dims=(224,224),n_labels=5)
test_gen = data_gen_small(rgb
,mask,depth,node,liste=test_list,batch_size=1,dims=(224,224),n_labels=5)


input_shape = (224, 224, 9)
output_shape = (224, 224, 5)
def train_gen_wrapper():
    return data_gen_small(rgb, mask, depth, node, liste=train_list, batch_size=16, dims=(224, 224), n_labels=5)
def val_gen_wrapper():
    return data_gen_small(rgb, mask, depth, node, liste=val_list, batch_size=16, dims=(224, 224), n_labels=5)
def test_gen_wrapper():
    return data_gen_small(rgb, mask, depth, node, liste=test_list, batch_size=1, dims=(224, 224), n_labels=5)

dataset = tf.data.Dataset.from_generator(
    train_gen_wrapper,
    output_signature=(
        tf.TensorSpec(shape=(16, *input_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(16, *output_shape), dtype=tf.float32),
    )
)

val_dataset = tf.data.Dataset.from_generator(
    val_gen_wrapper,
    output_signature=(
        tf.TensorSpec(shape=(16, *input_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(16, *output_shape), dtype=tf.float32),
    )
)


test_dataset = tf.data.Dataset.from_generator(
    test_gen_wrapper,
    output_signature=(
        tf.TensorSpec(shape=(1, *input_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(1, *output_shape), dtype=tf.float32),
    )
)

In [ ]:
def dice_coef(y_true, y_pred):
    y_true = K.reshape(y_true, K.shape(y_pred))
    epsilon=1e-6
    axes = tuple(range(1, len(y_pred.shape)-1))
    numerator = 2. * K.sum(y_pred * y_true, axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)

    return K.mean((numerator + epsilon) / (denominator + epsilon))

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [ ]:
model.compile(loss=dice_coef_loss, optimizer=keras.optimizers.RMSprop(learning_rate=0.001), metrics=["accuracy", dice_coef])

In [ ]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/segnet/model_weights_unet_rms.weights.h5')

In [ ]:
log_dir = "/content/drive/MyDrive/Colab Notebooks/tensor_board"
profiler = tf.keras.callbacks.TensorBoard(log_dir = log_dir, profile_batch = (10,100),histogram_freq = 1 )

In [ ]:
dataset = dataset.prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
checkpoint_cb = ModelCheckpoint(
    filepath="/content/drive/MyDrive/Colab Notebooks/segnet/model_weights_unet_rms.weights.h5", #"/content/drive/MyDrive/Colab Notebooks/segnet/model_weights.weights.h5",  #model_weights2.weights.h5
    save_best_only=True,
    save_weights_only=True,
    monitor="val_loss",
    mode="min"
)

history = model.fit(
    dataset,
    steps_per_epoch=100,
    epochs=20,
    validation_data=val_dataset,
    validation_steps=100,
    callbacks = [checkpoint_cb,profiler]
)


In [ ]:
plt.plot(history.history['accuracy'],markersize=20)
plt.plot(history.history['val_accuracy'], markersize=20)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# plotting of training and validation loss curves
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
def convert_exr_to_rgb(exr_path):
    exr_file = OpenEXR.InputFile(exr_path)

    header = exr_file.header()
    data_window = header['dataWindow']
    width = data_window.max.x - data_window.min.x + 1
    height = data_window.max.y - data_window.min.y + 1

    channels = ['R', 'G', 'B']
    rgb_data = [np.frombuffer(exr_file.channel(channel, Imath.PixelType(Imath.PixelType.FLOAT)), dtype=np.float32)
                for channel in channels]

    rgb_array = [np.clip(channel.reshape(height, width) * 255, 0, 255).astype(np.uint8) for channel in rgb_data]
    rgb_image = np.dstack(rgb_array)
    return rgb_image

In [ ]:
h=0
for i in train_list:
    if h<2:

        img_path = rgb[i]
        original_img = read_exr(img_path)

        plt.figure(figsize=(15,15))
        plt.subplot(1,2,1)
        plt.imshow(original_img)
        array_img=img_to_array(original_img)
        depth_path = depth[i]
        depth_img = read_exr(depth_path)

        node_path = node[i]
        node_img = read_exr(node_path)

        res_img = np.concatenate((array_img,depth_img,node_img),axis=-1)
        res_img = np.expand_dims(res_img, axis=0)
        y_pred=model.predict(res_img)
        y_pred = np.reshape(y_pred[0], (224, 224, 5))
        c=class_pixels(y_pred)
        o=colorize(c)
        plt.subplot(1,2,2)
        plt.imshow(o)
        plt.show()
        h+=1
        print("Original Image Shape:", array_img.shape)
        print("Predicted Shape:", y_pred.shape)
        print("Class Pixels Shape:", c.shape)
        print("Colorized Output Shape:", o.shape)

    else:
        break